In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import time
%matplotlib inline
pd.set_option('display.max_columns',120)
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [ ]:
!dir -l {data_dir}

dir: cannot access '{data_dir}': No such file or directory


In [ ]:
train=pd.read_csv('/content/drive/MyDrive/train____.csv')

In [ ]:
sample_size=0.1

import random
random.seed(123)

dtypes = {
    'fare_amount': 'float32',
    'pickup_longitude': 'float32',
    'pickup_latitude': 'float32',
    'dropoff_longitude': 'float32',
    'passenger_count': 'float32'
}
train=pd.read_csv('/content/drive/MyDrive/train____.csv',
               skiprows=lambda i: i>0 and random.random() > sample_size,dtype=dtypes
)
print("before dropping NA:{}".format(train.isna().sum()))
df=train.dropna()
print("before dropping NA:{}".format(train.isna().sum()))
train.describe()

In [ ]:
test=pd.read_csv("/content/test.csv")
print("before dropping NA:{}".format(train.isna().sum()))
test=test.dropna()
print("before dropping NA:{}".format(train.isna().sum()))
test.describe()

In [ ]:
train.info()

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9914 entries, 0 to 9913
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   key                9914 non-null   object 
 1   pickup_datetime    9914 non-null   object 
 2   pickup_longitude   9914 non-null   float64
 3   pickup_latitude    9914 non-null   float64
 4   dropoff_longitude  9914 non-null   float64
 5   dropoff_latitude   9914 non-null   float64
 6   passenger_count    9914 non-null   int64  
dtypes: float64(4), int64(1), object(2)
memory usage: 542.3+ KB


In [ ]:
def remove_outliers(train):
    return train[(train['fare_amount'] >= 1.) &
              (train['fare_amount'] <= 50.) &
              (train['pickup_longitude'] >= -75) &
              (train['pickup_longitude'] <= -72) &
              (train['dropoff_longitude'] >= -75) &
              (train['dropoff_longitude'] <= -72) &
              (train['pickup_latitude'] >= 40) &
              (train['pickup_latitude'] <= 41) &
              (train['dropoff_latitude'] >=40) &
              (train['dropoff_latitude'] <= 41) &
              (train['passenger_count'] >= 1) &
              (train['passenger_count'] <= 4)]
train=remove_outliers(train)

In [ ]:
train.describe()

In [ ]:
plt.hist(train['fare_amount'], bins=20)
plt.xlabel('Fare Amount')
plt.ylabel('Frequency')
plt.title('Distribution of Fare Amount')
plt.show()

In [ ]:
sampled_data = train.sample(n=10000)
plt.scatter(sampled_data['pickup_longitude'], sampled_data['pickup_latitude'], label='Pickup')
plt.scatter(sampled_data['dropoff_longitude'], sampled_data['dropoff_latitude'], label='Dropoff')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Pickup and Dropoff Locations')
plt.legend()
plt.show()

In [ ]:
passenger_counts = train['passenger_count'].value_counts()
plt.bar(passenger_counts.index, passenger_counts.values)
plt.xlabel('Passenger Count')
plt.ylabel('Frequency')
plt.title('Distribution of Passenger Counts')
plt.show()

In [ ]:
train.to_parquet('train.parquets')
test.to_parquet('test.parquets')

In [ ]:
train.columns

Index(['key', 'fare_amount', 'pickup_datetime', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count'],
      dtype='object')

In [ ]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(train, test_size=0.2, random_state=123)

In [ ]:
len(train_df), len(val_df)

(3871776, 967944)

In [ ]:
train_df = train_df.dropna()
val_df = val_df.dropna()

In [ ]:
train.columns

Index(['key', 'fare_amount', 'pickup_datetime', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count'],
      dtype='object')

In [ ]:
input_cols = ['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'passenger_count']
target_cols='fare_amount'

In [ ]:
train_inputs=train[input_cols]
train_inputs

In [ ]:
train_targets = train[target_cols]
train_targets

In [ ]:
val_inputs = val_df[input_cols]
val_inputs

In [ ]:
val_targets = val_df[target_cols]
val_targets

In [ ]:
test_inputs = test[input_cols]
test_inputs

In [ ]:
train_inputs = train_df[input_cols]
train_targets = train_df[target_col]
val_inputs = val_df[input_cols]
val_targets = val_df[target_col]
test_inputs = test[input_cols]

In [ ]:
def rmse(model):
    model.fit(train_inputs, train_targets)
    train_preds=model.predict(train_inputs)
    val_preds=model.predict(val_inputs)
    train_rmse = mean_squared_error(train_targets, train_preds, squared=False)
    val_rmse = mean_squared_error(val_targets, val_preds, squared=False)
    return train_rmse,val_rmse

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
linear_regression = LinearRegression()

rmse(linear_regression)


In [ ]:
class MeanRegressor:
    def __init__(self):
        self.mean = None

    def fit(self, inputs, targets):
        self.mean = targets.mean()

    def predict(self, inputs):
        return np.full(inputs.shape[0], self.mean)

In [ ]:
mean_model = MeanRegressor()
mean_model.fit(train_inputs, train_targets)

mean_model.mean

10.68132

In [ ]:
train_predicts = mean_model.predict(train_inputs)
train_predicts

array([10.68132, 10.68132, 10.68132, ..., 10.68132, 10.68132, 10.68132],
      dtype=float32)

In [ ]:
train_targets

In [ ]:
val_predicts = mean_model.predict(val_inputs)
val_predicts

array([10.68132, 10.68132, 10.68132, ..., 10.68132, 10.68132, 10.68132],
      dtype=float32)

In [ ]:
val_targets

In [ ]:
def rmse(targets, preds):
    return mean_squared_error(targets, preds, squared = False)

In [ ]:
train_rmse = rmse(train_targets, train_predicts)
train_rmse

7.7081404

In [ ]:
val_rmse = rmse(val_targets, val_predicts)
val_rmse

7.6884565

In [ ]:
linear_model = LinearRegression()

In [ ]:
linear_model.fit(train_inputs, train_targets)

LinearRegression()

In [ ]:
train_preds = linear_model.predict(train_inputs)
train_preds

array([ 8.47039187, 10.13884244,  9.53849297, ..., 11.90286835,
        8.7986418 , 14.85847064])

In [ ]:
train_targets

In [ ]:
rmse( train_targets, train_preds )

6.6887104571773675

In [ ]:
val_preds = linear_model.predict(val_inputs)
val_preds

array([ 9.82145548, 11.25412748,  8.42732919, ...,  9.63447714,
       10.92478371, 12.23083747])

In [ ]:
rmse(val_targets, val_preds )

6.670265404193188

In [ ]:
test_inputs

In [ ]:
submission = pd.DataFrame()
submission['key'] = test['key']
submission['fare_amount'] = 11.35

submission.to_csv('sample_submission.csv', index=False)